In [1]:
# 📍 1. Imports
import pandas as pd
from pathlib import Path

In [ ]:
# 📍 2. Load Labeled Data
df = pd.read_csv("../data/processed/labeled_messages.csv")
def clean_date(x):
    """Convert '########' or empty strings to NaT, else parse M/D/YYYY."""
    if isinstance(x, str) and x.strip().startswith("#"):
        return pd.NaT
    try:
        return pd.to_datetime(x, format="%m/%d/%Y", errors="coerce")
    except Exception:
        return pd.NaT

df["date_parsed"] = df["date"].apply(clean_date)
#save parsed date column
df.to_csv("../data/processed/labeled_messages.csv", index=False)
df = df.dropna(subset=["date_parsed"])  # Drop rows without date

In [4]:
sentiment_map = {"Positive": 1, "Negative": -1, "Neutral": 0}
df["Sentiment_Score"] = df["Sentiment"].map(sentiment_map)

The monthly sentiment score directly aggregates labeled values with no weighting or scaling.

In [5]:
df["YearMonth"] = df["date_parsed"].dt.to_period("M")

In [6]:
monthly_scores = (
    df.groupby(["from", "YearMonth"])["Sentiment_Score"]
    .sum()
    .reset_index()
    .rename(columns={"from": "Employee", "YearMonth": "Month", "Sentiment_Score": "Score"})
)

In [ ]:
print(monthly_scores.head())
monthly_scores.to_csv("../data/processed/monthly_sentiment_scores.csv", index=False)

                      Employee    Month  Score
0  bobette.riner@ipgdirect.com  2010-01      2
1  bobette.riner@ipgdirect.com  2010-02      7
2  bobette.riner@ipgdirect.com  2010-03      4
3  bobette.riner@ipgdirect.com  2010-04      2
4  bobette.riner@ipgdirect.com  2010-05      1
